In [1]:
import ee
import geemap.core as geemap

from google.colab import drive


import numpy as np
import pandas as pd

from scipy.interpolate import griddata
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import requests
import math
import io
import sys


# Mount Google Drive
drive.mount('/content/drive')


# File path to your JSON file in Google Drive
private_key = '/content/drive/My Drive/Thesis_Coding/ee-thesis-project-5752d6e0c694.json'

# Authenticating Earth Engine
service_account = 'mailto:notebook@ee-thesis-project.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, private_key)
ee.Initialize(credentials)

Mounted at /content/drive


In [2]:
sys.setrecursionlimit(5000)

In [3]:
demRaw = ee.Image("USGS/SRTMGL1_003")#ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').mean()
dsmRaw = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')
landformsRaw = ee.Image('CSP/ERGo/1_0/Global/SRTM_landforms')
hydroRaw = ee.Image('MERIT/Hydro/v1_0_1')
gsw = ee.Image('JRC/GSW1_3/GlobalSurfaceWater')
climRaw = ee.Image('WORLDCLIM/V1/BIO')
vegRaw = ee.ImageCollection('MODIS/006/MOD44B').mean()
# biomassRaw = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY').mean()
bedrockRaw = ee.Image("ISDASOIL/Africa/v1/bedrock_depth")
densityRaw = ee.Image("ISDASOIL/Africa/v1/bulk_density")
carbonRaw = ee.Image("ISDASOIL/Africa/v1/carbon_total")
fccRaw = ee.Image("ISDASOIL/Africa/v1/fcc")
phRaw = ee.Image("ISDASOIL/Africa/v1/ph")
clayRaw = ee.Image("ISDASOIL/Africa/v1/clay_content")
sandRaw = ee.Image("ISDASOIL/Africa/v1/sand_content")
siltRaw = ee.Image("ISDASOIL/Africa/v1/silt_content")

In [4]:
def get_rectangle_bounds(lon, lat, distance):
  """
  This function takes in longitude, latitude, and a distance in meters, and returns the bounds of a rectangle centered at the given point, with sides of length equal to the distance, suitable for ee.Geometry.Rectangle.

  Args:
    lon: The longitude of the center point.
    lat: The latitude of the center point.
    distance: The distance in meters from the center point to each edge of the rectangle.

  Returns:
    A list of four floats representing the bounds of the rectangle: [xMin, yMin, xMax, yMax].
  """

  # Convert the distance to degrees
  distance_deg = distance / 111132  # Assuming Earth is a sphere with radius 6371 km

  # Calculate the offsets in longitude and latitude
  lon_offset = distance_deg * math.cos(math.radians(lat))
  lat_offset = distance_deg

  # Calculate rectangle bounds
  x_min = lon - lon_offset
  y_min = lat - lat_offset
  x_max = lon + lon_offset
  y_max = lat + lat_offset

  return [x_min, y_min, x_max, y_max]

In [5]:
def get_site_data(lat, lon, bufferDistance):
  # buffer = ee.Geometry.Point(lon, lat).buffer(bufferDistance)
  buffer = ee.Geometry.Rectangle(get_rectangle_bounds(lon,lat,bufferDistance))

  dem = demRaw.select('elevation')
  filteredDem = dem.clip(buffer)

  proj = dsmRaw.first().select(0).projection()
  slope = ee.Terrain.slope(dsmRaw.mosaic().setDefaultProjection(proj)).clip(buffer)

  landforms = landformsRaw.select('constant')
  filteredLandforms = landforms.clip(buffer)

  hydro = hydroRaw.select('dir')
  filteredHydro = hydro.clip(buffer)

  point = ee.Geometry.Point([lon, lat])
  water_mask = gsw.gt(0).selfMask()
  gsw_distance = water_mask.mask().fastDistanceTransform(neighborhood=1024).multiply(ee.Image.pixelArea()).sqrt().rename('gsw_distance')
  gswDistance = gsw_distance.reduceRegion(ee.Reducer.first(),
                                                    point, water_mask.projection().nominalScale()).getNumber('gsw_distance').getInfo()

  clim = climRaw.clip(buffer)
  bio01 = clim.select('bio01')
  bio03 = clim.select('bio03')
  bio12 = clim.select('bio12')

  veg = vegRaw.clip(buffer)
  treeCover = veg.select('Percent_Tree_Cover').reduceRegion(ee.Reducer.mean()).getInfo()
  treeCover = treeCover[list(treeCover)[0]]
  nonTree = veg.select('Percent_NonTree_Vegetation').reduceRegion(ee.Reducer.mean()).getInfo()
  nonTree = nonTree[list(nonTree)[0]]

  # biomass = biomassRaw.clip(buffer)
  # agbd = biomass.select('agbd').reduceRegion(ee.Reducer.mean())

  bedrock = bedrockRaw.clip(buffer).select('mean_0_200')

  density = densityRaw.clip(buffer).select('mean_20_50')

  carbon = carbonRaw.clip(buffer).select('mean_20_50')

  fcc = fccRaw.clip(buffer).select('fcc')

  ph = phRaw.clip(buffer).select('mean_20_50')

  clay = clayRaw.clip(buffer).select('mean_20_50')

  sand = sandRaw.clip(buffer).select('mean_20_50')

  silt = siltRaw.clip(buffer).select('mean_20_50')

  return {'coor': [lat, lon], 'buffer': buffer, 'dem': filteredDem, 'slope': slope, 'landforms': filteredLandforms,
          'hydro': filteredHydro, 'gswDistance': gswDistance, 'bio01': bio01, 'bio03': bio03, 'bio12': bio12,
          'treeCover': treeCover, 'nonTree': nonTree, #'agbd': agbd,
          'bedrock': bedrock, 'density': density, 'carbon': carbon,
          'fcc': fcc, 'ph': ph, 'clay': clay, 'sand': sand, 'silt': silt}

In [6]:
def get_array(img):
  url = img.getDownloadUrl({'format': 'NPY'})
  response = requests.get(url)
  data = np.load(io.BytesIO(response.content)).astype(np.float32)
  return data


In [7]:
def resample_nearest_neighbor(input_array, output_shape):
    # Generate input coordinates
    input_x, input_y = np.meshgrid(np.arange(input_array.shape[1]), np.arange(input_array.shape[0]))

    # Generate output coordinates
    output_x, output_y = np.meshgrid(np.linspace(0, input_array.shape[1]-1, output_shape[1]),
                                      np.linspace(0, input_array.shape[0]-1, output_shape[0]))

    # Reshape input and output coordinates for interpolation
    input_coords = np.vstack((input_x.ravel(), input_y.ravel())).T
    output_coords = np.vstack((output_x.ravel(), output_y.ravel())).T

    # Perform nearest neighbor interpolation
    output_array = griddata(input_coords, input_array.ravel(), output_coords, method='nearest')

    # Reshape output to desired shape
    output_array = output_array.reshape(output_shape)

    return output_array

In [9]:
def scale_ndarray(data):
    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Reshape the data to 2D array if it's not already
    if len(data.shape) > 2:
        data = data.reshape(-1, data.shape[-1])

    # Fit and transform the data
    scaled_data = scaler.fit_transform(data)

    # Reshape the scaled data back to the original shape
    scaled_data = scaled_data.reshape(data.shape)

    return scaled_data

In [10]:
def one_hot_encode(data):
  """
  Encodes categorical data using one-hot encoding.

  Args:
    data: A 2D numpy array containing categorical data.

  Returns:
    A 3D numpy array containing the one-hot encoded data.
  """

  unique_values = np.unique(data)
  encoded_data = np.zeros((data.shape[0], data.shape[1], len(unique_values)))

  for i in range(data.shape[0]):
    for j in range(data.shape[1]):
      val = data[i, j]
      idx = np.where(unique_values == val)[0][0]
      encoded_data[i, j, idx] = 1

  return encoded_data

In [11]:
def preprocess_data(siteData):
  ls = []
  for i in siteData:
    if i == 'dem':
      ls.append(scale_ndarray(get_array(siteData[i])))
    elif i == 'treeCover' or i == 'nonTree' or i == 'gswDistance':
      ls.append(siteData[i])
    elif i == 'landforms' or i == 'hydro':
      ls.append(one_hot_encode(resample_nearest_neighbor(get_array(siteData[i]), (649, 563))))
    elif i != 'coor' and i != 'buffer':
      ls.append(scale_ndarray(resample_nearest_neighbor(get_array(siteData[i]), (649, 563))))
  return ls

In [11]:
# x = get_site_data(24.9733,32.8707,10000)
# y = preprocess_data(x)

In [12]:

# Get the geometry of Egypt
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
egypt = countries.filter(ee.Filter.eq('ADM0_NAME', "Egypt"))
egypt_geometry = egypt.geometry()

random_points = ee.FeatureCollection.randomPoints(
    region=egypt_geometry, points=50, seed=0, maxError=1
)


In [13]:
sites = pd.read_csv(('/content/drive/My Drive/sites_coords.csv'), header=None)
sites.columns = ['ind','name', 'lat', 'lon']
sites = sites.drop('ind', axis=1)

In [14]:
geo_list = []
for ind, i in sites.iterrows():
  coords = [dict(i)['lon'], dict(i)['lat']]
  point = ee.Geometry.Point(coords).buffer(10000)
  geo_list.append(point)

sitesB5k = ee.FeatureCollection(geo_list)

def randpoints():
  points = ee.FeatureCollection.randomPoints(egypt_geometry, 1310, 123)
  points = points.geometry()
  points = points.difference(sitesB5k)
  return points.getInfo()['coordinates']

negatives = randpoints()
nonSites = pd.DataFrame(negatives)
dummy = pd.DataFrame(np.full(1180, 'None Site'))
nonSites = pd.concat([dummy, nonSites], axis=1)
nonSites.columns = ['name', 'lat', 'lon']

In [25]:
sitesDatals = []
nonDatals = []


for ind, i in sites.iterrows():
  if ind < 500:
    sitesDatals.append(get_site_data(list(i)[1], list(i)[2], 10000))
  else:
    break

In [34]:
for ind, i in sites.iterrows():
  if ind >= 500 and ind < 1000:
    sitesDatals.append(get_site_data(list(i)[1], list(i)[2], 10000))
  elif ind < 1000:
    pass
  else:
    break

In [40]:
for ind, i in sites.iterrows():
  if ind >= 1000:
    sitesDatals.append(get_site_data(list(i)[1], list(i)[2], 10000))
  elif ind < 1180:
    pass
  else:
    break

In [43]:
for ind, i in nonSites.iterrows():
  if ind < 500:
    nonDatals.append(get_site_data(list(i)[1], list(i)[2], 10000))
  elif ind < 1000:
    pass
  else:
    break

In [45]:
for ind, i in nonSites.iterrows():
  if ind >= 500 and ind < 1000:
    nonDatals.append(get_site_data(list(i)[1], list(i)[2], 10000))
  elif ind < 1000:
    pass
  else:
    break

In [46]:
for ind, i in nonSites.iterrows():
  if ind >= 1000:
    nonDatals.append(get_site_data(list(i)[1], list(i)[2], 10000))
  elif ind < 1180:
    pass
  else:
    break

In [19]:
sitesData= []
nonSitesData = []

In [21]:
counter = [0]
for i in range(44):
  counter.append((counter[len(counter)-1])+27)

In [40]:
# ind = 0 #load pickled data to continue

for i in sitesDatals[counter[ind]: counter[ind+1]]:
  try:
    sitesData.append(preprocess_data(i))
    print(f"Site Appended @ {counter[ind]} ~ {counter[ind+1]}")
  except IndexError:
    print(counter[ind])

ind += 1
print(counter[ind], len(sitesData))

Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
Site Appended @ 81 ~ 108
108 108


In [38]:
print(counter[ind], len(sitesData))

54 54


In [ ]:
for i in sitesDatals[:ind]:
  sitesData.append(preprocess_data(i))

In [ ]:
for i in nonDatals:
  nonSitesData.append(preprocess_data(i))

In [18]:
print('positive example size: ',(sys.getsizeof(sitesDatals) / (1024 * 1024)),
      'Mgbs \n\n','negative exampls size: ',(sys.getsizeof(nonDatals) / (1024 * 1024)), 'Mgbs')

positive example size:  0.00905609130859375 Mgbs 

 negative exampls size:  0.00905609130859375 Mgbs


In [41]:
# # prompt: Use pickle to save sitesDatals and nonDatals to one drive

# import pickle

with open('/content/drive/My Drive/sitesData.pkl', 'wb') as f:
  pickle.dump(sitesData, f)

with open('/content/drive/My Drive/counter.pkl', 'wb') as f:
  pickle.dump(counter, f)

with open('/content/drive/My Drive/ind.pkl', 'wb') as f:
  pickle.dump(ind, f)


KeyboardInterrupt: 

In [13]:
# # prompt: Retrieve the two variables pickled in the last cell

# import pickle

# with open('/content/drive/My Drive/sites_data.pkl', 'rb') as f:
#   sitesDatals = pickle.load(f)

# with open('/content/drive/My Drive/non_sites_data.pkl', 'rb') as f:
#   nonDatals = pickle.load(f)


## next steps:
  4.   connect one drive
  5.   collect data for sites
  

